<a href="https://colab.research.google.com/github/mtavakoliangit/ena/blob/main/ena_1device_garage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pandas as pd
from datetime import datetime, date

In [3]:
#load data from gdrive
df_1dev_garage = pd.read_csv('gdrive/My Drive/ENA_db/garage_onedevice.csv')

In [4]:
df_1dev_garage.head()

,_id,day,device_id,esp_version,first,last,nsamples,reading_heat_time_q_max,reading_heat_time_q_min,reading_temperature_U6,samples,st_version
0,5faf735c9cf7b6988ae0e760,2020-11-13,09963013926712196740163043482,"[""0.0.45""]",23:04:24,23:59:21,43,39628,39628,8.0,"[{""time_stamp"":""23:04:24"",""setpoint_temperatur...","[""000.000.032""]"
1,5faf8079a183f18be8354312,2020-11-14,09963013926712196740163043482,"[""0.0.45""]",00:00:08,23:54:40,1331,46648,39628,8.5,"[{""time_stamp"":""00:00:08"",""setpoint_temperatur...","[""000.000.032""]"
2,5fb0d21da183f18be836844e,2020-11-15,09963013926712196740163043482,"[""0.0.45""]",00:00:44,23:59:39,1335,47309,46648,7.0,"[{""time_stamp"":""00:00:44"",""setpoint_temperatur...","[""000.000.032""]"
3,5fb22353a183f18be837d3db,2020-11-16,09963013926712196740163043482,"[""0.0.45""]",00:00:34,23:58:56,1332,50220,47309,8.0,"[{""time_stamp"":""00:00:44"",""setpoint_temperatur...","[""000.000.032""]"
4,5fb374f244ad208acea906b4,2020-11-17,09963013926712196740163043482,"[""0.0.45""]",00:00:00,23:59:31,1332,51868,50220,9.0,"[{""time_stamp"":""00:00:00"",""setpoint_temperatur...","[""000.000.032""]"


In [5]:
df_1dev_garage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      367 non-null    object 
 1   day                      367 non-null    object 
 2   device_id                367 non-null    object 
 3   esp_version              367 non-null    object 
 4   first                    367 non-null    object 
 5   last                     367 non-null    object 
 6   nsamples                 367 non-null    int64  
 7   reading_heat_time_q_max  367 non-null    int64  
 8   reading_heat_time_q_min  367 non-null    int64  
 9   reading_temperature_U6   367 non-null    float64
 10  samples                  367 non-null    object 
 11  st_version               367 non-null    object 
dtypes: float64(1), int64(3), object(8)
memory usage: 34.5+ KB


In [6]:
df_1dev_garage['device_id'].unique()

array(['09963013926712196740163043482'], dtype=object)

In [7]:
#filter data for a specific device
df_device = df_1dev_garage[df_1dev_garage['device_id'] == '09963013926712196740163043482']

In [8]:
df_device.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 367 entries, 0 to 366
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      367 non-null    object 
 1   day                      367 non-null    object 
 2   device_id                367 non-null    object 
 3   esp_version              367 non-null    object 
 4   first                    367 non-null    object 
 5   last                     367 non-null    object 
 6   nsamples                 367 non-null    int64  
 7   reading_heat_time_q_max  367 non-null    int64  
 8   reading_heat_time_q_min  367 non-null    int64  
 9   reading_temperature_U6   367 non-null    float64
 10  samples                  367 non-null    object 
 11  st_version               367 non-null    object 
dtypes: float64(1), int64(3), object(8)
memory usage: 37.3+ KB


In [9]:
df_device.describe()

,nsamples,reading_heat_time_q_max,reading_heat_time_q_min,reading_temperature_U6
count,367.000000,367.000000,367.000000,367.000000
mean,1183.136240,252665.504087,248741.607629,9.787466
std,509.011802,216929.856153,216820.698200,5.652124
min,1.000000,0.000000,0.000000,-1.500000
25%,1331.500000,4730.000000,4730.000000,6.000000
50%,1431.000000,231995.000000,226432.000000,7.500000
75%,1448.000000,456718.500000,456417.000000,11.000000
max,1511.000000,562996.000000,562996.000000,26.500000


In [10]:
df_device.head()

,_id,day,device_id,esp_version,first,last,nsamples,reading_heat_time_q_max,reading_heat_time_q_min,reading_temperature_U6,samples,st_version
0,5faf735c9cf7b6988ae0e760,2020-11-13,09963013926712196740163043482,"[""0.0.45""]",23:04:24,23:59:21,43,39628,39628,8.0,"[{""time_stamp"":""23:04:24"",""setpoint_temperatur...","[""000.000.032""]"
1,5faf8079a183f18be8354312,2020-11-14,09963013926712196740163043482,"[""0.0.45""]",00:00:08,23:54:40,1331,46648,39628,8.5,"[{""time_stamp"":""00:00:08"",""setpoint_temperatur...","[""000.000.032""]"
2,5fb0d21da183f18be836844e,2020-11-15,09963013926712196740163043482,"[""0.0.45""]",00:00:44,23:59:39,1335,47309,46648,7.0,"[{""time_stamp"":""00:00:44"",""setpoint_temperatur...","[""000.000.032""]"
3,5fb22353a183f18be837d3db,2020-11-16,09963013926712196740163043482,"[""0.0.45""]",00:00:34,23:58:56,1332,50220,47309,8.0,"[{""time_stamp"":""00:00:44"",""setpoint_temperatur...","[""000.000.032""]"
4,5fb374f244ad208acea906b4,2020-11-17,09963013926712196740163043482,"[""0.0.45""]",00:00:00,23:59:31,1332,51868,50220,9.0,"[{""time_stamp"":""00:00:00"",""setpoint_temperatur...","[""000.000.032""]"


In [11]:
df_device = df_device[['day', 'nsamples', 'samples']]

In [12]:
df_device.head()

,day,nsamples,samples
0,2020-11-13,43,"[{""time_stamp"":""23:04:24"",""setpoint_temperatur..."
1,2020-11-14,1331,"[{""time_stamp"":""00:00:08"",""setpoint_temperatur..."
2,2020-11-15,1335,"[{""time_stamp"":""00:00:44"",""setpoint_temperatur..."
3,2020-11-16,1332,"[{""time_stamp"":""00:00:44"",""setpoint_temperatur..."
4,2020-11-17,1332,"[{""time_stamp"":""00:00:00"",""setpoint_temperatur..."


In [13]:
df_device['day']= pd.to_datetime(df_device['day'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [87]:
data_comp = pd.DataFrame(columns=['DateTime', 'setpoint_temp', 'setpoint_fan', 'reading_temp'])

In [ ]:
for i in range(0, df_device.shape[0]):
  print(i)
  data_string = df_device['samples'][i].split()
  data_Series_format = pd.Series(data_string)
  data = data_Series_format.str.split(r'"time_stamp":"|","setpoint_temperature":|,"setpoint_fan":|,"reading_temperature_U6":|},{|}]', expand=True)
  counter = 1
  while counter <= data.shape[1] - 1:
    data[counter] = pd.to_datetime(data[counter],format= '%H:%M:%S' ).dt.time
    data[counter+1] = float(data[counter+1])
    data[counter+2] = float(data[counter+2])
    data[counter+3] = float(data[counter+3])
    counter = counter + 5
  j = 1
  n = 1
  while n < df_device['nsamples'][i]:
    dt = df_device['day'][i].replace(hour=data[j][0].hour, minute=data[j][0].minute, second=data[j][0].second)
    settemp = data[j+1][0]
    setfan = data[j+2][0]
    temp = data[j+3][0]
    data_comp = data_comp.append({'DateTime': dt, 'setpoint_temp': settemp, 'setpoint_fan': setfan, 'reading_temp': temp}, ignore_index=True)
    n = n + 1
    j = j + 5

0
1
2
3


In [ ]:
data_comp